In [1]:
import os
import pandas as pd
import re

In [12]:
def sample_type(barcode):
    # example: get "01" from TCGA-W5-A22X-01A
    # 00-09 tumor; 10-19 normal; 20-29 control
    match = re.search(r'-([0-2][0-9])[A-Z]*', barcode)
    if match:
        code = int(match.group(1))
        if code < 10:
            return "TUMOR"
        elif code < 20:
            return "NORMAL"
        elif code < 30:
            return "CONTROL"
        else:
            raise SyntaxError("Its not a TCGA barcode format.")

def open_vcf(vcf_file):
    # Find the line number containing string "#CHROM"
    with open(vcf_file, 'r') as f:
        for line_ind, line in enumerate(f):
            if line.startswith('#CHROM'):
                break
    
    # load data and skip headers
    df = pd.read_csv(vcf_file, sep='\t', header=line_ind, dtype=object)
    return df

def preprocessing_vcf(vcf_file):
    vcf_df = open_vcf(vcf_file)
    pass_filter = vcf_df.FILTER == "PASS"
    # print(pass_filter.sum(), pass_filter.count(), pass_ratio)
    vcf_df = vcf_df.loc[pass_filter] # keep "PASS"
    vcf_df = vcf_df.drop(columns=['ID', 'QUAL', 'FILTER'])#, 'FORMAT'])
    # keep only chr1-22 or XY
    vcf_df = vcf_df[vcf_df['#CHROM'].str.contains(r'^chr(?:[1-9]|1\d|2[0-2]|X|Y)$')]
    vcf_df.columns = ['CHROM'] + list(vcf_df.columns[1:].str.upper())

    # rename last 2 columns to tumor/normal
    #
    vcf_df.index = range(len(vcf_df))
    return vcf_df


In [7]:
import os
os.chdir("/home/data/dataset/CHOL_10sample/")
sample_sheet = pd.read_csv("./gdc_sample_sheet.2024-02-26.tsv", sep="\t")


In [13]:

# sample_sheet = pd.read_csv("/home/data/dataset/CHOL_2sample/gdc_sample_sheet_test.2024-02-26.tsv", sep="\t")
#case_ID
#for sample_ID in sample_sheet.:
#gb = sample_sheet.groupby("Case ID")
#for case_ID in gb.groups:
#    print(case_ID)

for case_ID in sample_sheet.loc[:, "Case ID"]:
    muse = preprocessing_vcf(vcf_file=f"variants_calling/muse/{case_ID}.MuSE.vcf")
    mutect2 = preprocessing_vcf(vcf_file=f"variants_calling/mutect_only_vcf/{case_ID}_filtered.vcf")
    mutect2 = mutect2.rename(columns={mutect2.columns[-1]: sample_type(mutect2.columns[-1]), 
                                      mutect2.columns[-2]: sample_type(mutect2.columns[-2])})
    varscan_indel = preprocessing_vcf(vcf_file=f"variants_calling/varscan_GDC/{case_ID}.varscan.indel.Somatic.hc.vcf")
    varscan_snp = preprocessing_vcf(vcf_file=f"variants_calling/varscan_GDC/{case_ID}.varscan.snp.Somatic.hc.vcf")
    varscan = pd.concat([varscan_indel, varscan_snp], ignore_index=True)
    # 合併邏輯

In [ ]:
def parse_info(package_name, callformat):
    '''
    varscan, GT:GQ:DP:RD:AD:FREQ:DP4, 0/0:.:68:65:2:2.99%:15,50,1,1
    mutect2, GT:AD:AF:DP:F1R2:F2R1, 0/1:12,4:0.275:16:8,2:4,2
    muse, GT:DP:AD:BQ:SS, 0/1:19:14,5:31,31:2	
    '''
    if package_name == "varscan"

In [17]:
format_str = "GT:GQ:DP:RD:AD:FREQ:DP4"


,CHROM,POS,REF,ALT,INFO,FORMAT,TUMOR,NORMAL
0,chr1,8967564,A,G,SOMATIC,GT:DP:AD:BQ:SS,"0/1:19:14,5:31,31:2","0/0:26:26,0:31,0:."
1,chr1,16045536,G,A,SOMATIC,GT:DP:AD:BQ:SS,"0/1:24:17,7:28,29:2","0/0:42:42,0:28,0:."
2,chr1,27550441,C,T,SOMATIC,GT:DP:AD:BQ:SS,"0/1:13:8,5:29,31:2","0/0:23:23,0:29,0:."
3,chr1,43981297,G,T,SOMATIC,GT:DP:AD:BQ:SS,"0/1:30:21,9:32,30:2","0/0:34:34,0:33,0:."
4,chr1,61762741,T,C,SOMATIC,GT:DP:AD:BQ:SS,"0/1:54:38,16:29,31:2","0/0:74:74,0:29,0:."
...,...,...,...,...,...,...,...,...
115,chr22,37206698,T,A,SOMATIC,GT:DP:AD:BQ:SS,"0/1:75:65,10:30,29:2","0/0:81:81,0:30,0:."
116,chr22,41589133,A,T,SOMATIC,GT:DP:AD:BQ:SS,"0/1:39:31,8:30,28:2","0/0:50:50,0:30,0:."
117,chrX,106903064,C,A,SOMATIC,GT:DP:AD:BQ:SS,"0/1:27:17,10:34,29:2","0/0:19:19,0:34,0:."
118,chrX,120604680,G,T,SOMATIC,GT:DP:AD:BQ:SS,"0/1:13:5,8:35,28:2","0/0:14:14,0:33,0:."


In [15]:
muse_format = muse.FORMAT[0].split(":")

DP_index = muse_format.index("DP*")
try:
    AF_index = muse_format.index("AF")
except ValueError:
    AD_index = muse_format.index("AD")
muse_format.index("DP")

ValueError: 'DP*' is not in list

In [58]:
muse.columns = ['CHROM'] + list(muse.columns[1:].str.upper())

In [23]:
gb.get_group("TCGA-W5-AA2X").loc[:, "Sample ID"].apply()


0    TCGA-W5-AA2X-01A
1    TCGA-W5-AA2X-11A
Name: Sample ID, dtype: object

In [9]:
mutect2.columns[-2:] = sample_type(mutect2.columns[-2]), sample_type(mutect2.columns[-1])


('tumor', 'normal')

In [ ]:
sample_ID = "TCGA-W5-AA2X-01A"	
mutect2.columns = mutect2.columns[:-2] + mutect2.columns[-2:]

In [5]:
mutect2, pass_ratio = preprocessing_vcf("mutect2/TCGA-W5-AA2X_filtered.vcf")
mutect2

,#CHROM,POS,REF,ALT,INFO,TCGA-W5-AA2X-01A,TCGA-W5-AA2X-11A
6,chr1,1390825,G,A,"CONTQ=53;DP=171;ECNT=1;GERMQ=402;MBQ=36,30;MFR...","0/1:49,3:0.074:52:21,0:28,3","0/0:113,0:8.570e-03:113:40,0:73,0"
12,chr1,2149147,G,T,"CONTQ=64;DP=12;ECNT=1;GERMQ=22;MBQ=35,34;MFRL=...","0/1:1,2:0.604:3:0,0:1,2","0/0:8,0:0.100:8:3,0:5,0"
13,chr1,2156164,G,T,"CONTQ=51;DP=19;ECNT=1;GERMQ=30;MBQ=35,34;MFRL=...","0/1:6,2:0.300:8:3,2:3,0","0/0:10,0:0.081:10:2,0:8,0"
17,chr1,3635943,C,T,"CONTQ=65;DP=94;ECNT=1;GERMQ=213;MBQ=37,32;MFRL...","0/1:24,3:0.137:27:11,0:13,3","0/0:64,0:0.015:64:18,0:46,0"
37,chr1,11957858,T,C,"CONTQ=93;DP=153;ECNT=1;GERMQ=334;MBQ=33,30;MFR...","0/1:26,13:0.341:39:10,3:16,10","0/0:111,0:8.789e-03:111:46,0:65,0"
...,...,...,...,...,...,...,...
7660,chrUn_JTFH01001430v1_decoy,222,T,G,"CONTQ=74;DP=38;ECNT=2;GERMQ=71;MBQ=33,35;MFRL=...","0/1:13,3:0.221:16:6,1:7,2","0/0:20,0:0.045:20:8,0:12,0"
7681,chrUn_JTFH01001478v1_decoy,500,G,C,"CONTQ=43;DP=33;ECNT=1;GERMQ=70;MBQ=36,32;MFRL=...","0/1:13,2:0.176:15:7,0:6,2","0/0:18,0:0.050:18:8,0:10,0"
7692,chrUn_JTFH01001553v1_decoy,782,T,C,"CONTQ=70;DP=13;ECNT=1;GERMQ=33;MBQ=33,36;MFRL=...","0/1:0,2:0.750:2:0,1:0,1","0/0:10,0:0.081:10:6,0:4,0"
7705,chrUn_JTFH01001847v1_decoy,1652,C,T,"CONTQ=87;DP=16;ECNT=1;GERMQ=22;MBQ=37,32;MFRL=...","0/1:5,3:0.400:8:3,2:2,1","0/0:8,0:0.100:8:2,0:6,0"


In [104]:
mutect2 = open_vcf("mutect2/TCGA-W5-AA2X_filtered.vcf")
pass_filter = mutect2.FILTER == "PASS"
pass_ratio = pass_filter.sum()/pass_filter.count()
print(pass_filter.sum(), pass_filter.count(), pass_ratio)

mutect2 = mutect2.loc[pass_filter] # keep "PASS"
mutect2 = mutect2.drop(columns=['ID', 'QUAL', 'FILTER', 'FORMAT'])

0.0471441523118767

In [105]:
sample_type(mutect2.columns[-1])

'normal'

In [13]:
muse = open_vcf("muse/TCGA-W5-AA2X.MuSE.vcf")
pass_filter = muse.FILTER == "PASS"
muse = muse.loc[pass_filter] # keep "PASS"
muse = muse.drop(columns=['ID', 'QUAL', 'FILTER'])
# drop CHROM not in 1-22 or XY
muse = muse[muse['#CHROM'].str.contains(r'^chr(?:[1-9]|1\d|2[0-2]|X|Y)$')]


In [14]:
muse

,#CHROM,POS,REF,ALT,INFO,FORMAT,TUMOR,NORMAL
3,chr1,2156164,G,T,SOMATIC,GT:DP:AD:BQ:SS,"0/1:8:6,2:35,34:2","0/0:10:10,0:35,0:."
9,chr1,11957858,T,C,SOMATIC,GT:DP:AD:BQ:SS,"0/1:50:33,17:31,28:2","0/0:144:140,3:30,2:."
24,chr1,26773569,A,G,SOMATIC,GT:DP:AD:BQ:SS,"0/1:14:6,8:33,36:2","0/0:66:66,0:32,0:."
33,chr1,33772581,G,A,SOMATIC,GT:DP:AD:BQ:SS,"0/1:11:5,6:37,27:2","0/0:14:14,0:36,0:."
40,chr1,45530932,C,A,SOMATIC,GT:DP:AD:BQ:SS,"0/1:10:7,3:37,34:2","0/0:20:20,0:34,0:."
...,...,...,...,...,...,...,...,...
1147,chrX,55221659,A,C,SOMATIC,GT:DP:AD:BQ:SS,"0/1:30:7,23:32,37:2","0/0:47:47,0:31,0:."
1154,chrX,72712853,A,G,SOMATIC,GT:DP:AD:BQ:SS,"1/1:28:4,24:31,33:2","0/0:37:37,0:30,0:."
1156,chrX,103724114,C,T,SOMATIC,GT:DP:AD:BQ:SS,"0/1:15:5,10:33,32:2","0/0:35:35,0:33,0:."
1177,chrY,12859165,A,T,SOMATIC,GT:DP:AD:BQ:SS,"0/1:65:56,9:31,30:2","0/0:121:121,0:30,0:."


In [ ]:
varscan_snp = open_vcf("muse/TCGA-W5-AA2X.MuSE.vcf")


In [94]:




#mutect2.columns[-1] = sample_type(mutect2.columns[-1])
#mutect2.columns[-2] = sample_type(mutect2.columns[-2])

# drop CHROM not in 1-22 or XY
mutect2 = mutect2[mutect2['#CHROM'].str.contains(r'^chr(?:[1-9]|1\d|2[0-2]|X|Y)$')]

# explode multihit - 一個REF對到多個ALT(少見)
multi_hit_filter = mutect2['ALT'].str.contains(',')
multi_hit_split = mutect2.loc[multi_hit_filter, 'ALT'].str.split(',')
mutect2.loc[multi_hit_filter, 'ALT'] = multi_hit_split
mutect2 = mutect2.explode('ALT')
mutect2 = mutect2.reset_index(drop=True)

In [97]:
mutect2.INFO[0]

'CONTQ=53;DP=171;ECNT=1;GERMQ=402;MBQ=36,30;MFRL=215,174;MMQ=60,60;MPOS=29;NALOD=2.06;NLOD=34.00;POPAF=6.00;SAAF=0.030,0.051,0.058;SAPP=0.028,5.545e-03,0.967;TLOD=5.48'

In [92]:
mutect2_copy = mutect2[multi_hit_filter].copy()
mutect2_copy

/tmp/ipykernel_3605354/702046631.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  mutect2_copy = mutect2[multi_hit_filter].copy()


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [77]:
mutect2_copy = mutect2[multi_hit_filter].copy()
mutect2_copy["ALT"] = mutect2_copy["ALT"].str.split(',')
mutect2_copy.explode('ALT')

,#CHROM,POS,REF,ALT,INFO,FORMAT,TCGA-W5-AA2X-01A,TCGA-W5-AA2X-11A
1506,chr3,167683840,CATAT,C,"CONTQ=43;DP=64;ECNT=1;GERMQ=56,0;MBQ=37,35,38;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:13,3,2:0.181,0.145:18:3,1,1:10,2,1","0/0:26,0,3:0.027,0.123:29:13,0,1:13,0,2"
1506,chr3,167683840,CATAT,CAT,"CONTQ=43;DP=64;ECNT=1;GERMQ=56,0;MBQ=37,35,38;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:13,3,2:0.181,0.145:18:3,1,1:10,2,1","0/0:26,0,3:0.027,0.123:29:13,0,1:13,0,2"
2306,chr6,36603076,ATT,A,"CONTQ=25;DP=47;ECNT=1;GERMQ=14,0;MBQ=28,31,24;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:11,3,3:0.189,0.201:17:4,1,1:7,2,2","0/0:14,0,1:0.047,0.105:15:5,0,1:9,0,0"
2306,chr6,36603076,ATT,AT,"CONTQ=25;DP=47;ECNT=1;GERMQ=14,0;MBQ=28,31,24;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:11,3,3:0.189,0.201:17:4,1,1:7,2,2","0/0:14,0,1:0.047,0.105:15:5,0,1:9,0,0"
3639,chr10,5458438,TAA,T,"CONTQ=93;DP=94;ECNT=1;GERMQ=2,104;MBQ=33,33,35...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:10,2,5:0.126,0.281:17:3,0,1:7,2,4","0/0:42,3,1:0.075,0.031:46:11,0,1:31,3,0"
3639,chr10,5458438,TAA,TAAA,"CONTQ=93;DP=94;ECNT=1;GERMQ=2,104;MBQ=33,33,35...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:10,2,5:0.126,0.281:17:3,0,1:7,2,4","0/0:42,3,1:0.075,0.031:46:11,0,1:31,3,0"
4820,chr14,33801862,GAAA,G,"CONTQ=73;DP=73;ECNT=1;GERMQ=93,0;MBQ=35,37,36;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:12,2,3:0.148,0.198:17:3,1,1:9,1,2","0/0:32,0,8:0.021,0.201:40:13,0,5:19,0,3"
4820,chr14,33801862,GAAA,GAA,"CONTQ=73;DP=73;ECNT=1;GERMQ=93,0;MBQ=35,37,36;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:12,2,3:0.148,0.198:17:3,1,1:9,1,2","0/0:32,0,8:0.021,0.201:40:13,0,5:19,0,3"


ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [53]:
multi_hit_split

,#CHROM,POS,REF,ALT,INFO,FORMAT,TCGA-W5-AA2X-01A,TCGA-W5-AA2X-11A
1506,chr3,167683840,CATAT,"C,CAT","CONTQ=43;DP=64;ECNT=1;GERMQ=56,0;MBQ=37,35,38;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:13,3,2:0.181,0.145:18:3,1,1:10,2,1","0/0:26,0,3:0.027,0.123:29:13,0,1:13,0,2"
2306,chr6,36603076,ATT,"A,AT","CONTQ=25;DP=47;ECNT=1;GERMQ=14,0;MBQ=28,31,24;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:11,3,3:0.189,0.201:17:4,1,1:7,2,2","0/0:14,0,1:0.047,0.105:15:5,0,1:9,0,0"
3639,chr10,5458438,TAA,"T,TAAA","CONTQ=93;DP=94;ECNT=1;GERMQ=2,104;MBQ=33,33,35...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:10,2,5:0.126,0.281:17:3,0,1:7,2,4","0/0:42,3,1:0.075,0.031:46:11,0,1:31,3,0"
4820,chr14,33801862,GAAA,"G,GAA","CONTQ=73;DP=73;ECNT=1;GERMQ=93,0;MBQ=35,37,36;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:12,2,3:0.148,0.198:17:3,1,1:9,1,2","0/0:32,0,8:0.021,0.201:40:13,0,5:19,0,3"


1506     [C, CAT]
2306      [A, AT]
3639    [T, TAAA]
4820     [G, GAA]
Name: ALT, dtype: object

In [44]:
multi_hit_filter = mutect2['ALT'].str.contains(',')
multi_hit_split = mutect2[multi_hit_filter].copy()

mutect2[multi_hit_filter, 'ALT'] = mutect2_split['ALT'].str.split(',')


TypeError: unhashable type: 'Series'

In [38]:
mutect2_split['ALT'].str.split(',')

,#CHROM,POS,REF,ALT,INFO,FORMAT,TCGA-W5-AA2X-01A,TCGA-W5-AA2X-11A
1506,chr3,167683840,CATAT,C,"CONTQ=43;DP=64;ECNT=1;GERMQ=56,0;MBQ=37,35,38;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:13,3,2:0.181,0.145:18:3,1,1:10,2,1","0/0:26,0,3:0.027,0.123:29:13,0,1:13,0,2"
1506,chr3,167683840,CATAT,CAT,"CONTQ=43;DP=64;ECNT=1;GERMQ=56,0;MBQ=37,35,38;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:13,3,2:0.181,0.145:18:3,1,1:10,2,1","0/0:26,0,3:0.027,0.123:29:13,0,1:13,0,2"
2306,chr6,36603076,ATT,A,"CONTQ=25;DP=47;ECNT=1;GERMQ=14,0;MBQ=28,31,24;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:11,3,3:0.189,0.201:17:4,1,1:7,2,2","0/0:14,0,1:0.047,0.105:15:5,0,1:9,0,0"
2306,chr6,36603076,ATT,AT,"CONTQ=25;DP=47;ECNT=1;GERMQ=14,0;MBQ=28,31,24;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:11,3,3:0.189,0.201:17:4,1,1:7,2,2","0/0:14,0,1:0.047,0.105:15:5,0,1:9,0,0"
3639,chr10,5458438,TAA,T,"CONTQ=93;DP=94;ECNT=1;GERMQ=2,104;MBQ=33,33,35...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:10,2,5:0.126,0.281:17:3,0,1:7,2,4","0/0:42,3,1:0.075,0.031:46:11,0,1:31,3,0"
3639,chr10,5458438,TAA,TAAA,"CONTQ=93;DP=94;ECNT=1;GERMQ=2,104;MBQ=33,33,35...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:10,2,5:0.126,0.281:17:3,0,1:7,2,4","0/0:42,3,1:0.075,0.031:46:11,0,1:31,3,0"
4820,chr14,33801862,GAAA,G,"CONTQ=73;DP=73;ECNT=1;GERMQ=93,0;MBQ=35,37,36;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:12,2,3:0.148,0.198:17:3,1,1:9,1,2","0/0:32,0,8:0.021,0.201:40:13,0,5:19,0,3"
4820,chr14,33801862,GAAA,GAA,"CONTQ=73;DP=73;ECNT=1;GERMQ=93,0;MBQ=35,37,36;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:12,2,3:0.148,0.198:17:3,1,1:9,1,2","0/0:32,0,8:0.021,0.201:40:13,0,5:19,0,3"


In [34]:
 mutect2[mutect2['ALT'].str.contains(',')].copy()

,#CHROM,POS,REF,ALT,INFO,FORMAT,TCGA-W5-AA2X-01A,TCGA-W5-AA2X-11A
1506,chr3,167683840,CATAT,"C,CAT","CONTQ=43;DP=64;ECNT=1;GERMQ=56,0;MBQ=37,35,38;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:13,3,2:0.181,0.145:18:3,1,1:10,2,1","0/0:26,0,3:0.027,0.123:29:13,0,1:13,0,2"
2306,chr6,36603076,ATT,"A,AT","CONTQ=25;DP=47;ECNT=1;GERMQ=14,0;MBQ=28,31,24;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:11,3,3:0.189,0.201:17:4,1,1:7,2,2","0/0:14,0,1:0.047,0.105:15:5,0,1:9,0,0"
3639,chr10,5458438,TAA,"T,TAAA","CONTQ=93;DP=94;ECNT=1;GERMQ=2,104;MBQ=33,33,35...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:10,2,5:0.126,0.281:17:3,0,1:7,2,4","0/0:42,3,1:0.075,0.031:46:11,0,1:31,3,0"
4820,chr14,33801862,GAAA,"G,GAA","CONTQ=73;DP=73;ECNT=1;GERMQ=93,0;MBQ=35,37,36;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:12,2,3:0.148,0.198:17:3,1,1:9,1,2","0/0:32,0,8:0.021,0.201:40:13,0,5:19,0,3"


In [33]:
mutect2_split

,#CHROM,POS,REF,ALT,INFO,FORMAT,TCGA-W5-AA2X-01A,TCGA-W5-AA2X-11A
1506,chr3,167683840,CATAT,C,"CONTQ=43;DP=64;ECNT=1;GERMQ=56,0;MBQ=37,35,38;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:13,3,2:0.181,0.145:18:3,1,1:10,2,1","0/0:26,0,3:0.027,0.123:29:13,0,1:13,0,2"
1506,chr3,167683840,CATAT,CAT,"CONTQ=43;DP=64;ECNT=1;GERMQ=56,0;MBQ=37,35,38;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:13,3,2:0.181,0.145:18:3,1,1:10,2,1","0/0:26,0,3:0.027,0.123:29:13,0,1:13,0,2"
2306,chr6,36603076,ATT,A,"CONTQ=25;DP=47;ECNT=1;GERMQ=14,0;MBQ=28,31,24;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:11,3,3:0.189,0.201:17:4,1,1:7,2,2","0/0:14,0,1:0.047,0.105:15:5,0,1:9,0,0"
2306,chr6,36603076,ATT,AT,"CONTQ=25;DP=47;ECNT=1;GERMQ=14,0;MBQ=28,31,24;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:11,3,3:0.189,0.201:17:4,1,1:7,2,2","0/0:14,0,1:0.047,0.105:15:5,0,1:9,0,0"
3639,chr10,5458438,TAA,T,"CONTQ=93;DP=94;ECNT=1;GERMQ=2,104;MBQ=33,33,35...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:10,2,5:0.126,0.281:17:3,0,1:7,2,4","0/0:42,3,1:0.075,0.031:46:11,0,1:31,3,0"
3639,chr10,5458438,TAA,TAAA,"CONTQ=93;DP=94;ECNT=1;GERMQ=2,104;MBQ=33,33,35...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:10,2,5:0.126,0.281:17:3,0,1:7,2,4","0/0:42,3,1:0.075,0.031:46:11,0,1:31,3,0"
4820,chr14,33801862,GAAA,G,"CONTQ=73;DP=73;ECNT=1;GERMQ=93,0;MBQ=35,37,36;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:12,2,3:0.148,0.198:17:3,1,1:9,1,2","0/0:32,0,8:0.021,0.201:40:13,0,5:19,0,3"
4820,chr14,33801862,GAAA,GAA,"CONTQ=73;DP=73;ECNT=1;GERMQ=93,0;MBQ=35,37,36;...",GT:AD:AF:DP:F1R2:F2R1,"0/1/2:12,2,3:0.148,0.198:17:3,1,1:9,1,2","0/0:32,0,8:0.021,0.201:40:13,0,5:19,0,3"
